In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cvxpy as cp

## Generate the random Dataset


In [2]:
# We first generate a random dataset with number of features (m = 10) and number of instances (n = 100)
# We also generate a random label vector y \in {-1,1}

n = 100 # Number of instances
m = 10  # Number of Features

X = np.random.rand(n,m)
y = np.random.rand(n) # n-dimensional vector
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m) # m-dimensional vector
print(y)
print(X)

[-1  1 -1  1  1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1  1  1  1 -1 -1 -1  1 -1
  1 -1  1  1 -1 -1  1  1  1 -1 -1  1  1  1 -1 -1 -1 -1  1  1 -1 -1  1 -1
  1 -1  1  1  1 -1  1  1 -1  1  1  1 -1  1  1 -1  1 -1 -1  1 -1 -1  1 -1
 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1  1  1  1  1 -1  1 -1 -1 -1
 -1  1 -1  1]
[[1.95308134e-01 7.98993353e-01 4.75689550e-01 8.95398497e-01
  1.07940165e-01 9.03779672e-01 5.72325321e-01 5.75167111e-01
  5.77609176e-01 2.59268131e-02]
 [4.01920901e-01 7.17031473e-01 8.22095305e-01 4.98535230e-01
  6.84010833e-01 8.99425187e-01 2.37969460e-01 4.38075804e-01
  9.09224164e-01 3.29484319e-01]
 [9.30846308e-01 1.90408500e-01 6.16041421e-01 9.00011847e-01
  6.13387883e-01 7.15934359e-01 1.30317855e-01 7.50406615e-02
  3.45122745e-01 9.17114738e-01]
 [8.76874822e-01 2.37296844e-01 6.79455538e-01 8.88564704e-01
  2.01282615e-02 5.93740516e-01 6.11518228e-01 5.68840805e-01
  9.27045654e-01 8.45448183e-01]
 [5.02701746e-01 2.72156381e-01 7.59799136e-01 3.25204401e-01
  1.

## An Implementation of the Logistic Loss 


In [3]:
def LogisticLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += np.log(1 + np.exp(-y[i] * y_pred[i] / Z) + eps)
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            g[k] += -1 / Z * y[i] * X[i][k] / (1 + np.exp(y_pred[i] * y[i] / Z))
            
        g[k] += lam * w[k]
    
    return [f, g]     

In [4]:
start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.00404810905456543
Function value = 70.7717943578683
Printing Gradient:
[0.44762367 0.08973384 0.31585168 0.40553753 0.86898475 0.95854302
 0.04003747 0.09037728 0.82529141 0.27722872]


## An Implementation of the Least Squares 


In [5]:
def LeastSquaresNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += (y[i] - y_pred[i]) ** 2
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            g[k] += 2 * (y_pred[i] - y[i]) * X[i][k]
            
        g[k] += lam * w[k]
        
    return [f, g]     

In [6]:
start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.0013513565063476562
Function value = 592.4418387511391
Printing Gradient:
[218.78777935 223.94225295 207.13676927 230.3816916  233.02343169
 229.52959471 203.56834853 216.14121597 225.23050176 231.09580668]


## An Implementation of the Hinge Loss 

In [7]:
def HingeLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += max(0, 1 - y[i] * y_pred[i])
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            if y_pred[i]*y[i] <= 1:
                g[k] += -1 * y[i] * X[i][k]
            
        g[k] += lam * w[k]
    
    return [f, g]

In [8]:
start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.002384662628173828
Function value = 162.0859208106271
Printing Gradient:
[25.18041011 28.14926623 23.18463409 27.56012733 27.42301974 24.89255245
 24.98899065 25.32120959 26.64823403 27.18671608]


## Scalability of the code

In [9]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Logistic Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Least Square")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Hinge Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Logistic Loss
Time Taken = 3.898196220397949
Function value = [1796.54727871]
Printing Gradient:
[0.8400511  0.18189239 0.80721711 ... 0.02633613 0.36537191 1.00021811]
Least Square
Time Taken = 2.2098374366760254
Function value = [6.27662628e+08]
Printing Gradient:
[244757.08304822 254686.46205992 249341.92352536 ... 237448.77319858
 245402.92231102 254711.99661506]
Hinge Loss
Time Taken = 2.2023420333862305
Function value = [114355.70860497]
Printing Gradient:
[23.74126879 23.25239655 21.58371545 ... 18.64416456 23.35025911
 23.26455111]


## Implement a vectorized version 

In [10]:
def LogisticLossVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    y_pred = X @ w
    f = np.sum(np.log(1 + np.exp(-y_pred * y / Z) + eps)) + lam * np.linalg.norm(w) / 2
    g = - X.T / Z @ (y / (1 + np.exp(y * y_pred / Z))) + lam * w
    return [f, g]     

In [11]:
def LeastSquaresVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    y_pred = X @ w
    f = np.sum((y_pred - y) ** 2) + lam * np.linalg.norm(w) / 2
    g = 2 * X.T @ (y_pred - y) + lam * w
    return [f, g]     

In [12]:
def HingeLossVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    y_pred = X @ w
    f = np.sum((1 >= y_pred * y) * (1 - y_pred*y)) + lam * np.linalg.norm(w) / 2
    g = - X.T @ ((1 >= y_pred * y) * y) + lam * w
    return [f, g]

In [13]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossVec(w,X,y,1)
end = time.time()
print("Logistic Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = LeastSquaresVec(w,X,y,1)
end = time.time()
print("Least Square")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = HingeLossVec(w,X,y,1)
end = time.time()
print("Hinge Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Logistic Loss
Time Taken = 0.01532602310180664
Function value = 12526.878008560307
Printing Gradient:
[[0.68991544 0.73797091 0.73797091 ... 0.68991544 0.68991544 0.73797091]
 [0.49476298 0.54481732 0.54481732 ... 0.49476298 0.49476298 0.54481732]
 [0.99288557 1.04056825 1.04056825 ... 0.99288557 0.99288557 1.04056825]
 ...
 [0.38752089 0.43914399 0.43914399 ... 0.38752089 0.38752089 0.43914399]
 [0.43486326 0.48552706 0.48552706 ... 0.43486326 0.43486326 0.48552706]
 [0.47084739 0.51805012 0.51805012 ... 0.47084739 0.47084739 0.51805012]]
Least Square
Time Taken = 0.011134862899780273
Function value = 61982037399.058685
Printing Gradient:
[[239246.47191507 239438.69375845 239438.69375845 ... 239246.47191507
  239246.47191507 239438.69375845]
 [249244.78500667 249445.00236645 249445.00236645 ... 249244.78500667
  249244.78500667 249445.00236645]
 [237091.93462334 237282.66532823 237282.66532823 ... 237091.93462334
  237091.93462334 237282.66532823]
 ...
 [257111.25747339 257317.7498867

## Lets us code the above Loss Fuctions in CVXPY!

CVXPY is an open source Python-embedded modeling language for convex optimization problems. Link: https://www.cvxpy.org/

In [14]:
def LogisticLossCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    expression = cp.sum(cp.log(1 + cp.exp(cp.multiply(-y, X @ w / Z)) + eps)) + lam * cp.norm(w, 2) / 2
    Problem = cp.Problem(cp.Minimize(expression))
    f = expression.value
    g = - X.T / Z @ (y / (1 + np.exp(y * (X @ w) / Z))) + lam * w
    
    result = Problem.solve()
    print(result)
    return [f, g]

In [15]:
def LeastSquaresCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    expression = cp.sum_squares((X @ w) - y) + lam * cp.norm(w, 2) / 2
    Problem = cp.Problem(cp.Minimize(expression))
    f = expression.value
    g = 2 * X.T @ ((X @ w) - y) + lam * w
    
    result = Problem.solve()
    print(result)
    return [f, g]

In [16]:
def HingeLossCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    expression = cp.sum(cp.pos(1 - cp.multiply(y, X @ w))) + lam * cp.norm(w, 2) / 2
    Problem = cp.Problem(cp.Minimize(expression))
    f = expression.value
    g = - X.T @ ((1 >= (X @ w) * y) * y) + lam * w
    
    result = Problem.solve()
    print(result)
    return [f, g]

In [17]:
import numpy as np
n = 100
m = 10

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m)

start = time.time()
[f1,g1] = LogisticLossCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value Naive = " + str(f1))
print("Printing Gradient Naive:")
print(g1)

start = time.time()
[f2,g2] = LeastSquaresCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value For = " + str(f2))
print("Printing Gradient For:")
print(g2)

start = time.time()
[f2,g2] = HingeLossCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value For = " + str(f2))
print("Printing Gradient For:")
print(g2)

70.21460182595393
Time Taken = 0.0022406578063964844
Function value Naive = 70.21460182595393
Printing Gradient Naive:
[0.13117278 0.88521648 0.67992077 0.79986368 0.46976144 0.01714045
 0.61592522 0.08380271 0.86977947 0.18002219]
660.935095774148
Time Taken = 0.0006556510925292969
Function value For = 660.935095774148
Printing Gradient For:
[224.16526672 241.08679636 223.91489436 233.56189177 243.81953599
 211.72779837 255.79942849 258.12841761 280.71813562 229.49904144]
154.7038512226803
Time Taken = 0.001323699951171875
Function value For = 154.7038512226803
Printing Gradient For:
[22.1607932  22.33024964 22.11988189 20.93164732 22.54999438 21.7243705
 25.34241544 26.53443661 27.98528822 22.71092706]


## Compare the losses with Graph



In [ ]:
def LogisticLossFun(w, X, y, lam):
    return error_ll

def LeastSquaresFun(w, X, y, lam):
    return error_ls

def HingeLossFun(w, X, y, lam):
    return error_hl

def plot_errors(error_ll, error_ls, error_hl, num):
    plt.plot(num, error_ll, label="Logistic Loss")
    plt.plot(num, error_ls, label="Least Squares")
    plt.plot(num, error_hl, label="Hinge Loss")
    plt.show()
    return

In [ ]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

error_ll = LogisticLossFun(w,X,y,1)
error_ls = LeastSquaresFun(w,X,y,1)
error_hl = HingeLossFun(w,X,y,1)
plot_errors(error_ll, error_ls, error_hl, 100)